# 1) Data cleaning.

## Import the libraries.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import plotly.express as px

%matplotlib inline

# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

## Load the data.

In [ ]:
raw_df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
raw_df.head()


We can see that TotalCharges column is object datatype. We'll have to convert it to float datatype.

In [ ]:
raw_df.info()

In [ ]:
raw_df["TotalCharges"] = pd.to_numeric(raw_df["TotalCharges"],errors='coerce')

After conversion to float, we see that Total charges has 11 null values. We'll drop these null values.

In [ ]:
raw_df.isnull().sum()

In [ ]:
raw_df.dropna(inplace=True)

In [ ]:
raw_df.info()

We see that columns like MultipleLines, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV and StreamingMovies have values like "No Phone Services" and "No internet service". Converting these values to "No" as they don't contribute to any meaningful categorization and converting to "No" won't affect their relation as well as analysis.

In [ ]:
raw_df.value_counts('MultipleLines')
raw_df.value_counts('OnlineSecurity')
raw_df.value_counts('OnlineBackup')
raw_df.value_counts('DeviceProtection')
raw_df.value_counts('TechSupport')
raw_df.value_counts('StreamingTV')
raw_df.value_counts('StreamingMovies')

In [ ]:
raw_df['MultipleLines'].replace(to_replace="No phone service", value="No", inplace=True)
raw_df['OnlineSecurity'].replace(to_replace="No internet service", value="No", inplace=True)
raw_df['OnlineBackup'].replace(to_replace="No internet service", value="No", inplace=True)
raw_df['DeviceProtection'].replace(to_replace="No internet service", value="No", inplace=True)
raw_df['TechSupport'].replace(to_replace="No internet service", value="No", inplace=True)
raw_df['StreamingTV'].replace(to_replace="No internet service", value="No", inplace=True)
raw_df['StreamingMovies'].replace(to_replace="No internet service", value="No", inplace=True)

Removing the customer_id column as it serves no purpose for the customer churn analysis.

In [ ]:
new_df = raw_df.iloc[:,1:]

We can also notice that the for many rows, the (monthly charges * tenure) is not equal to total charges. Let's dive into. 

In [ ]:
new_df['calculatedCharges'] = new_df['tenure']*new_df['MonthlyCharges']

In [ ]:
#define conditions
conditions =[new_df['TotalCharges'] > new_df['calculatedCharges'], new_df['TotalCharges'] < new_df['calculatedCharges']]

#define choices
choices = ['greater', 'lesser']

new_df['TotalChargesStatus'] = np.select(conditions, choices, default='equal')
new_df

Here we can see that the total charges are not exactly computed as (tenure X Monthly charges) in the dataset. As per our calculations, we can see that only 614 transactions are equal to the above logic, the rest of the transactions are either greater or lesser in value. This means that the telco company has provided discounts for 3214 transactions and charged more(payment delay fines, or other value add charges) to 3204 transactions.

This seems deliberate and hence we won't be making any new changes to the TotalCharges column and leave it as is in the dataset. No data cleaning will be applied for this.

In [ ]:
new_df.value_counts('TotalChargesStatus')

In [ ]:
new_df.drop(new_df.columns[[-1,-2]], axis=1, inplace=True)
new_df

In [ ]:
#Convertin the predictor variable in a binary numeric variable. This is helpful if we are to build a model to predict churn.
new_df['Churn'].replace(to_replace='Yes', value=1, inplace=True)
new_df['Churn'].replace(to_replace='No',  value=0, inplace=True)
clean_df = new_df
clean_df

In [ ]:
#Exporting the clean_df.
clean_df.to_csv(r'C:\Users\samru\Documents\Coding\data_analysis_projects\DataAnalysisProjects\CustomerChurnAnalysis\docs\cleaned_dataset\cage_telco_customer_churn_clean.csv', index=False, header=True)

# 2) Data Exploration.

In [ ]:
#Let's convert all the categorical variables into dummy variables to find the correlation.
df_dummies = pd.get_dummies(clean_df)
df_dummies.head()

We can see factors like month-to-month contract, Fiber-optic internet service, payment-method electronic_check, monthly charges, paperlessBilling_yes, online security_no, no tech support, senior citizens, etc are positively correlated with churn, which means that with increase in these factors, higher churn.

Factors like tenure, two year contract, TotalCharges, OnlineSecurity_yes and TechSupport_yes have negative correlation.

In [ ]:
#Get Correlation of "Churn" with other variables:
plt.figure(figsize=(15,8))
df_dummies.corr()['Churn'].sort_values(ascending = False).plot(kind='bar')

## Explore demographics

Let us first understand the gender, age range, patner and dependent status of the customers

### Gender

We can see the genders are balanced, both male and female customers make about 50% each in the dataset. Gender also doesn't have much affect on churn as seen in the correlation chart.

In [ ]:
colors = ['#900C3F', '#F25900']
clean_df.gender.value_counts(normalize=True).mul(100).plot(xlabel="Gender", ylabel="% Customers", kind="bar", rot=0, title="Gender distribution in percentage(%)", color=colors)

Distribution of SeniorCitizen in the dataset. About 84% of customers are not senior citizens. Many young people in the dataset.

SeniorCitizen has a positive correlation with churn.

In [ ]:
clean_df.SeniorCitizen.value_counts(normalize=True).mul(100).plot.pie(autopct='%.2f%%', labels=['No', 'Yes'], figsize=(5,5), fontsize=12)

Distribution of gender amongst the senior citizens is 50-50% male to female. Similar distibution for non-senior citizen customers too.

In [ ]:
colors = ['#900C3F', '#F25900']
# clean_df.groupby(['SeniorCitizen', 'gender']).size().unstack().plot(kind="bar", stacked=True, rot=0, color = colors)
# clean_df.groupby(['SeniorCitizen', 'gender']).size().unstack()

temp_one = clean_df.groupby('SeniorCitizen')['gender'].value_counts(normalize=True, sort=False).mul(100).round(2)
temp_one.unstack().plot(ylabel="Customers in percentage(%)",kind="bar", stacked=True, rot=0, color=colors, title="SeniorCitizen vs gender stacked")
clean_df.groupby('SeniorCitizen')['gender'].value_counts(normalize=False, sort=False)

### Partner

We can see that about 50% of customers have dependents.

In [ ]:
clean_df.Partner.value_counts()

### Dependents

Around 70% of the customers do not have dependents. The Dependents_No does have a positive correlation with churn which has to be explored during churn analysis.

In [ ]:
colors = ['#FFC300', '#B1E73F']
clean_df.Dependents.value_counts(normalize=True).mul(100).round(2).plot(xlabel="Dependents", ylabel="Customers in percentage(%)", kind="bar", rot=0, title="Dependents distribution in percentage(%)", color=colors)

Let us explore partner-dependents relationship

In [ ]:
colors = ['#FFC300', '#B1E73F']
temp_one = clean_df.groupby('Partner')['Dependents'].value_counts(normalize=True, sort=False).mul(100).round(2)
temp_one.unstack().plot(kind="bar", ylabel="customers in percentage(%)", stacked=True, rot=0, color=colors, title="Partner vs Dependents relation in percentage(%)")
clean_df.groupby('Partner')['Dependents'].value_counts(normalize=False, sort=False)

## Explore Customer Account details.

Let us look into tenure and contract types and their relation with each other.

### Tenure

Majority of the customers have tenure less than 1-3 months as seen on the hist plot. The second peak shows us that customers who are loyal or long time have stayed with the telco company for more that 70 months. This distribution has two peaks.

In [ ]:
ax = sb.distplot(clean_df['tenure'], hist=True, kde=False, bins=60, color='#3FA5E7', hist_kws={'edgecolor':'black'})
ax.set_ylabel("# of Customers")
ax.set_title("# of customers according to the tenure")

### Contract types.

Majority of the customers have taken the month-to-month contracts. The two-year and one-year contracts have almost the same no. of takers.

In [ ]:
colors=['#3F51E7', '#813FE7', '#E73FA5']
clean_df.Contract.value_counts().plot(kind="bar", rot=0, ylabel="# of customers", title="# No of customers vs contract", color=colors)

Let's see how each contract type performs in the given tenure.

Most of the month-to-month contracts last for 1-2 months and the two-year contracts, huge number of them last 70+ months. This indicates that long term(two-year, one-year) contracts customers are more loyal and less likely to churn compared to the monthly contract customers.

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, sharey = True, figsize = (20,6))

ax = sb.distplot(clean_df[clean_df['Contract']=='Month-to-month']['tenure'],
                   hist=True, kde=False,
                   bins=60, color = '#3F51E7',
                   hist_kws={'edgecolor':'black'},
                   kde_kws={'linewidth': 4},
                 ax=ax1)
ax.set_ylabel('# of Customers')
ax.set_xlabel('Tenure (months)')
ax.set_title('Month to Month Contract')

ax = sb.distplot(clean_df[clean_df['Contract']=='One year']['tenure'],
                   hist=True, kde=False,
                   bins=60, color = '#813FE7',
                   hist_kws={'edgecolor':'black'},
                   kde_kws={'linewidth': 4},
                 ax=ax2)
ax.set_xlabel('Tenure (months)',size = 14)
ax.set_title('One Year Contract',size = 14)

ax = sb.distplot(clean_df[clean_df['Contract']=='Two year']['tenure'],
                   hist=True, kde=False,
                   bins=60, color = '#E73FA5',
                   hist_kws={'edgecolor':'black'},
                   kde_kws={'linewidth': 4},
                 ax=ax3)

ax.set_xlabel('Tenure (months)')
ax.set_title('Two Year Contract')

## Explore Services provided by the telco company

Some of these services have a positive correlation with churn and that will be explored during churn analysis.

In [ ]:
services = ['PhoneService','MultipleLines','InternetService','OnlineSecurity',
           'OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']
colors=['#9FE2BF', '#CCCCFF', '#DE3163']

fig, axes = plt.subplots(nrows = 3,ncols = 3,figsize = (15,12))
for i, item in enumerate(services):
    if i < 3:
        ax = clean_df[item].value_counts().plot(kind = 'bar',ax=axes[0,i],rot = 0, color=colors)
        
    elif i >=3 and i < 6:
        ax = clean_df[item].value_counts().plot(kind = 'bar',ax=axes[1,i-3],rot = 0, color=colors)
        
    elif i < 9:
        ax = clean_df[item].value_counts().plot(kind = 'bar',ax=axes[2,i-6],rot = 0, color=colors)
    ax.set_title(item)

### Monthly and totalcharges and their relation

As monthly charge increase, we see an increase in total charge too.

In [ ]:
clean_df[['MonthlyCharges', 'TotalCharges']].plot.scatter(x = 'MonthlyCharges',y='TotalCharges')

# 3) Churn Analysis

Let's look at how many customers have churned in this dataset. 1 is "Yes" and 0 is "No".

We can see that around 27% customers have churned. Now let us find out the factors leading to this churn rate.

We will examine the factors that have positive correlation with churn and dig deep into it. Refer to the corr() plot.

In [ ]:
colors=['#00FFBF', '#F1D809']
clean_df.Churn.value_counts(normalize=True).mul(100).plot(kind="bar",xlabel="Churn",ylabel="Customers in percentage(%)",  rot=0, color=colors, title="Customer Churn in percentage(%)")
clean_df.Churn.value_counts()

## Gender and Senior Citizens

First let's confirm that gender doesn't have a strong influence in churning.

We can see that amongst the customers who have churned, around 26% are male and 27% are female. This is almost same in terms of distribution which leads us to conclude that customer churn for this telco company is not a gender issue.

In [ ]:
colors = ['#900C3F', '#F25900']
temp_two = clean_df.groupby('gender')['Churn'].value_counts(normalize=True, sort=False).mul(100).round(2)
temp_two.unstack().plot(kind="bar", ylabel="Customers in percentage(%)", stacked=True, rot=0, color=colors, title="Gender vs Churn distribution in percentage(%)")
clean_df.groupby('gender')['Churn'].value_counts(normalize=False, sort=False)

We know that the dataset consists of 83.8% non-senior citizens and the rest being senior citizens. Let's see how the churning is distributed amongst these two people category.

We can see from below about 74.5% of the churned customers are non-senior citizens. This means that the telco company is losing more younger customers and the senior citizens have been loyal.

In [ ]:
colors = ['#900C3F', '#F25900']
temp_three = clean_df.groupby('Churn')['SeniorCitizen'].value_counts(normalize=True, sort=False).mul(100).round(2)
temp_three.unstack().plot(kind="bar", ylabel="Customers in percentage(%)", stacked=True, rot=0, color=colors, title="Churn vs SeniorCitizen in percentage(%)")
clean_df.groupby('Churn')['SeniorCitizen'].value_counts(normalize=False, sort=False)

### Senior Citizen
Let's see amongst those who have churned, what is the distribution of SeniorCitizen with respect to Contract types.

We can see that amongst the churned customers, most had Month-to-month contract with 1214 non-senior citizens.

In [ ]:
temp_churn_yes_df = clean_df[(clean_df['Churn']== 1)]
temp_churn_yes_df.groupby('Contract')['SeniorCitizen'].value_counts(normalize=False, sort=False).unstack().plot(kind="bar", stacked=True, ylabel="# of customers", rot=0, title="Contract vs Senior Citizen with Churn=Yes")
temp_churn_yes_df.groupby('Contract')['SeniorCitizen'].value_counts(normalize=False, sort=False)

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook"

We can see that most of the non-senior citizens are churning with total charges less than 2000. This trend is also similar with senior citizens who are churning having total charges < 3000.

In [ ]:
px.box(data_frame=clean_df,x="SeniorCitizen",y="TotalCharges",color="Churn")

![TotalCharge_SeniorCitizen_Churn](images/TotalCharges_SeniorCitizen_Churn.png)

### Dependents

We can see that churning is high amongst the  customers who have no dependents. 82.5% of the customers who have churned have no Dependents. This was also observed in the correlation plot. Does the having no dependents make it easier for customers to churn or shift companies?

In [ ]:
colors = ['#FFC300', '#B1E73F']
temp_four = clean_df.groupby('Churn')['Dependents'].value_counts(normalize=False, sort=False)
temp_four.unstack().plot(kind="bar", ylabel="# Customers", stacked=True, rot=0, color=colors, title="Churn vs Dependents")
clean_df.groupby('Churn')['Dependents'].value_counts(normalize=True, sort=False).mul(100).round(2).astype(str) + '%'

## Customer Account details

## Churn vs. Tenure

We can see from the below plot that those who churn, their tenure in average is less than 10 months. Either company hasn't provided enough value for the customers within this short period for them to stay.

Long time customers have mostly stayed with an average of 40 months. We can also see customers with tenure greater than 60 to 70 months.

In [ ]:
sb.boxplot(x = clean_df.Churn, y = clean_df.tenure)

Let's see how the contract types compare with tenure and churn.

As we can see, the two year contract type has the highest tenure(60-70 +months) for customers who have remained loyal and not churned.

Maximum churning can be seen in the 1-3 months under month-to-month contract type.

In [ ]:
colors=['#3F51E7','#813FE7','#E73FA5']
plt.figure(figsize=(10,3),dpi=200)
sb.displot(data=clean_df,x='tenure',bins=60,col='Contract',row='Churn', color='#813FE7')

## Services provided by the telco company.

### Internet service
From the corr() plot, we could see that Fiber optic internet service has a positive correlation with Churn. Let's dig into that.

We can observe that amongst the churned customers, 69% of them had a Fiber optic Internet Service. This could mean that this Fiber Optic service provided by the telco company is not providing value to the customers.

In [ ]:
colors=['#9FE2BF', '#CCCCFF', '#DE3163']
temp_five = clean_df.groupby('Churn')['InternetService'].value_counts(normalize=True, sort=False).mul(100).round(2)
temp_five.unstack().plot(kind="bar", ylabel="Customers in percentage(%)", stacked=True, rot=0, color=colors, title="Churn vs InternetService in percentage(%)")
clean_df.groupby('Churn')['InternetService'].value_counts(normalize=False, sort=False)

Churned customers who had opted Fiber optic service had an average tenure of 12 months.

In [ ]:
px.box(data_frame=clean_df,x="InternetService",y="tenure",color="Churn")

![tenure_InternetService_Churn](images/tenure_InternetService_Churn.png)

## Payment Method

From the corr() plot, we can see the Electronic Check payment method has a positive correlation with churn. Let's analyze that.

Amongst the payment types, 57.3% of the churned customers had the Electronic check option. There might be a problem with the electronic check transfer that is causing payment related problems leaving customers unsatisfied.

In [ ]:
colors=['#9FE2BF', '#CCCCFF', '#DE3163', '#F1D809']
temp_five = clean_df.groupby('Churn')['PaymentMethod'].value_counts(normalize=True, sort=False).mul(100).round(2)
temp_five.unstack().plot(kind="bar", ylabel="Customers in percentage(%)", stacked=True, rot=0, color=colors, title="Churn vs PaymentMethod in percentage(%)")
clean_df.groupby('Churn')['PaymentMethod'].value_counts(normalize=False, sort=False)

## Tech Support

TechSupport has a positive correlation with churn as seen in the corr() plot.

From the plot below, we can see that despite large number of customers opting for no Tech support in both Churned and non-churned category; amongst the churned customers, we see 83.4% of them had opted no tech support. 

We need to ask a bigger question as to whether bundling the tech support(and improving the tech support experience) with month-to-month contract could help in retaining customers.

In [ ]:
colors=['#9FE2BF', '#CCCCFF', '#DE3163']
temp_five = clean_df.groupby('Churn')['TechSupport'].value_counts(normalize=True, sort=False).mul(100).round(2)
temp_five.unstack().plot(kind="bar", ylabel="Customers in percentage(%)", stacked=True, rot=0, color=colors, title="Churn vs TechSupport in percentage(%)")
clean_df.groupby('Churn')['TechSupport'].value_counts(normalize=False, sort=False)

Customers who have stayed with the company for longer tenure have opted for Tech Support. This could mean that either the long term contacts(one or two-year contracts) have been bundled with the tech support and it has provided value add. 

We also see the non-churned customers with smaller tenure(indicating month-to-month contract) have no tech support. Maybe introducing some form of tech support to month-to-month contracts could help retain customers because those who have churned have no tech support.

In [ ]:
plt.figure(figsize=(10,2),dpi=200)
sb.displot(data=clean_df,x='tenure',bins=60,col='TechSupport',row='Churn', color='#E73FA5')

# Monthly and Total Charges w.r.t Churn.

Here people who have more monthly charge instead of having lower total charge is more likely to churn.

In [ ]:
plt.figure(figsize=(10,4),dpi=200)
sb.scatterplot(data= clean_df,x='MonthlyCharges',y='TotalCharges',hue='Churn', linewidth=0.5,alpha=0.5,palette='Dark2')